In [1]:
import os
from dotenv import load_dotenv
from crewai.tools import BaseTool
import yfinance as yf
from crewai import Agent
from crewai_tools import ScrapeWebsiteTool
from crewai import Task
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

load_dotenv()

# Get your actual OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# Set your model name separately
model_name = "gpt-4o-mini"


class Tools:
    class StockPriceTool(BaseTool):
        name: str = "One month stock price history"
        description: str = (
            "Useful to get a month's worth of stock price data as CSV. The input of this tool should a ticker, for example AAPL, NET, TSLA etc..."
        )

        def _run(self, ticker: str) -> str:
            stock = yf.Ticker(ticker)
            return stock.history(period="1mo").to_csv()

    class StockNewsTool(BaseTool):
        name: str = "Stock news URLs"
        description: str = (
            "Useful to get URLs of news articles related to a stock. The input to this tool should be a ticker, for example 066570.KS, AAPL, NET."
        )

        def _run(self, ticker: str) -> list:
            stock = yf.Ticker(ticker)
            news = stock.news
            if not news:
                return "No news articles found for this ticker"
            return [article.get("link", "No link available") for article in news]

    class IncomeStatementTool(BaseTool):
        name: str = "Company's income statement"
        description: str = (
            "Useful to get the income statement of a stock as CSV. The input to this tool should be a ticker, for example AAPL, NET."
        )

        def _run(self, ticker: str) -> str:
            stock = yf.Ticker(ticker)
            return stock.income_stmt.to_csv()

    class BalanceSheetTool(BaseTool):
        name: str = "Balance sheet"
        description: str = (
            "Useful to get the balance sheet of a stock as CSV. The input to this tool should be a ticker, for example 066570.KS."
        )

        def _run(self, ticker: str) -> str:
            stock = yf.Ticker(ticker)
            return stock.balance_sheet.to_csv()

    class InsiderTransactionsTool(BaseTool):
        name: str = "Get insider transactions"
        description: str = (
            "Useful to get insider transactions of a stock as CSV. The input to this tool should be a ticker, for example AAPL, NET."
        )

        def _run(self, ticker: str) -> str:
            stock = yf.Ticker(ticker)
            return stock.insider_transactions.to_csv()


class Agents:
    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.StockPriceTool(),
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefully and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.StockNewsTool(),
                ScrapeWebsiteTool(),
            ],
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.BalanceSheetTool(),
                Tools.IncomeStatementTool(),
                Tools.InsiderTransactionsTool(),
            ],
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )


class Tasks:
    def research(self, agent):
        return Task(
            description="""Research the company with ticker {company}. 
            Analyze recent news, developments, and market sentiment.
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            expected_output="A detailed report summarizing the company's recent news, developments, and market sentiment.",
        )

    def technical_analysis(self, agent):
        return Task(
            description="""Analyze the stock price movements, trends, and technical indicators for ticker {company}.
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            expected_output="A technical analysis report highlighting trends, support/resistance levels, and potential entry/exit points.",
        )

    def financial_analysis(self, agent):
        return Task(
            description="""Analyze the financial statements and health for company ticker {company}.
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            expected_output="A financial analysis report summarizing the company's income statement, balance sheet, and insider transactions.",
        )

    def investment_recommendation(self, agent, context_tasks):
        return Task(
            description="""Provide an investment recommendation for company ticker {company} based on all available information.
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            context_tasks=context_tasks,
            expected_output="A clear investment recommendation (e.g., Buy, Sell, Hold) with supporting rationale.",
        )


# Initialize agents and tasks
agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.financial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager,
    [
        research_task,
        technical_task,
        financial_task,
    ],
)

# Create and run the crew
crew = Crew(
    agents=[researcher, technical_analyst, financial_analyst, hedge_fund_manager],
    tasks=[research_task, technical_task, financial_task, recommend_task],
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(
        model_name="gpt-4o-mini",
        temperature=0.5,
        api_key=api_key,
    ),
    memory=True,
)

result = crew.kickoff(
    inputs=dict(
        company="066570.KS",
    ),
)

print(result)

LLM value is None
LLM value is None
LLM value is None
LLM value is None
LLM value is a string
# Agent: Crew Manager
## Task: Research the company with ticker 066570.KS. 
            Analyze recent news, developments, and market sentiment.
            The company ticker is a required input parameter that will be provided.


# Agent: Crew Manager
## Thought: I need to gather information about the company with the ticker 066570.KS to analyze recent news, developments, and market sentiment.
## Using tool: Stock news URLs
## Tool Input: 
"{\"ticker\": \"066570.KS\"}"
## Tool Output: 
['No link available', 'No link available', 'No link available', 'No link available', 'No link available', 'No link available', 'No link available', 'No link available', 'No link available', 'No link available']


# Agent: Crew Manager
## Final Answer: 
Recent news articles, developments, or market sentiment information regarding the company with ticker 066570.KS could not be found. It may be worthwhile to explo